In [1]:
# imports
import cv2
from pathlib import Path, PosixPath
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, f1_score
import natsort
from skimage.feature import greycomatrix, greycoprops

ImportError: cannot import name 'greycomatrix' from 'skimage.feature.texture' (/Users/gzanoni/dev/inteligencia-computacional/.venv/lib/python3.11/site-packages/skimage/feature/texture.py)

In [ ]:
def get_files(caminho_p):
    i=0
    train, test = list(), list()
    
    for pasta in sorted(Path(caminho_p).iterdir()):
      if pasta.is_dir(): 
        for file in sorted(Path(pasta).iterdir()):
            if file.is_file() and file.name.endswith(".bmp"):
              num = file.name.split(".")[0].split("_")[1]
              if int(num) > 18:
                test.append(str(file.absolute()))
              else:
                train.append(str(file.absolute()))
              i+=1
    print(f"numero de arquivos: ", i)
    return train, test
  
def get_classe(x):
  classe = list()
  for i in x:
    file = PosixPath(i)
    classe.append(file.name.split("_")[0])
  return classe

In [ ]:
#https://drive.google.com/drive/folders/1-3DJsfgM73-AHlkjn3-VUedZWnmllCud?usp=drive_link
train, test = get_files("../Base_BFL_256/")
train = natsort.natsorted(train) 
test = natsort.natsorted(test)
classes_train = get_classe(train)
classes_test = get_classe(test)
print(classes_train)
print(train)
print(test)
print(f"tamanho de train {len(train)}")
print(f"tamanho de test {len(test)}")
print(f"tamanho de classes train {len(classes_train)}")
print(f"tamanho de classes test {len(classes_test)}")


In [ ]:
# Função para calcular o LBP de uma imagem
def calculate_lbp(imagefile):
    image = cv2.imread(str(imagefile))
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # Converte a imagem para tons de cinza
    lbp_image = np.zeros_like(gray) # Cria uma matriz de zeros com as mesmas dimensões da imagem
    height, width = gray.shape

    for i in range(1, height - 1):
        for j in range(1, width - 1):
            center = gray[i, j] # Pega o pixel central
            binary_code = 0
            binary_code |= (gray[i - 1, j - 1] > center) << 7 # Desloca 7 bits para a esquerda
            binary_code |= (gray[i - 1, j] > center) << 6
            binary_code |= (gray[i - 1, j + 1] > center) << 5
            binary_code |= (gray[i, j + 1] > center) << 4
            binary_code |= (gray[i + 1, j + 1] > center) << 3
            binary_code |= (gray[i + 1, j] > center) << 2
            binary_code |= (gray[i + 1, j - 1] > center) << 1 # Desloca 1 bit para a esquerda
            binary_code |= (gray[i, j - 1] > center) << 0
            lbp_image[i, j] = binary_code # Armazena o valor do código binário na matriz

    return lbp_image


In [ ]:
def knn(neighbors, train, classes, test):

    knn = KNeighborsClassifier(n_neighbors=neighbors, metric='euclidean')
    knn.fit(train, classes)
    test_pred = knn.predict(test)
    return test_pred


In [ ]:
lbp_train = list()
for imagefile in train:
    lbp = calculate_lbp(imagefile)
    lbp_train.append(lbp.ravel())

lbp_test = list()
for imagefile in test:
    lbp = calculate_lbp(imagefile)
    lbp_test.append(lbp.ravel())

In [ ]:

test_pred = knn(3, lbp_train, classes_train, lbp_test)
print(test_pred)
print(classes_test)

In [ ]:
print(accuracy_score(classes_test, test_pred))
print(f1_score(classes_test, test_pred, average='macro'))

In [ ]:
# Calcular o LBP da imagem
lbp = calculate_lbp('../Base_BFL_256/CF00001/CF00001_1.bmp')

lbp_df = pd.DataFrame(lbp)
print(np.array(lbp.flatten()))
#escrever em arquivo
with open('lbp.txt', 'w') as f:
    # for item in lbp.flatten():
        f.write("%s\n" % lbp_df.to_string(index=False, header=False))

lbp_df

# Mostrar a imagem LBP
# cv2.imshow('LBP Image', lbp)
# cv2.waitKey(0)
# cv2.destroyAllWindows()

In [ ]:
def calculate_glcm(imagefile):
    glcm_features = []
    image = cv2.imread(str(imagefile), cv2.IMREAD_GRAYSCALE)
    glcm = greycomatrix(image, [1], [0], symmetric=True, normed=True)
    glcm_features.append(greycoprops(glcm, prop='contrast')[0][0])
    glcm_features.append(greycoprops(glcm, prop='dissimilarity')[0][0])
    glcm_features.append(greycoprops(glcm, prop='homogeneity')[0][0])
    glcm_features.append(greycoprops(glcm, prop='ASM')[0][0])
    glcm_features.append(greycoprops(glcm, prop='energy')[0][0])
    glcm_features.append(greycoprops(glcm, prop='correlation')[0][0])
    return glcm_features